In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt



df = pd.read_excel(r'01-Oct-2019_12_52_40_ay24087_Restreint4.xlsx', sheet_name='Restreint' )
dataset = df.values
X = dataset[1:,23:43]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')

##0,12,4,16
d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
d1.drop([1,2,3,5,6,8,9,10,11,13,14,15,17,18], axis=1, inplace=True)
d2.replace(0.  , np.nan, inplace=True)
d2.replace(-1. , np.nan, inplace=True)
d2.replace(1.  , np.nan, inplace=True)
d2.replace(2.  , np.nan, inplace=True)
d2.replace(3.  , np.nan, inplace=True)
d2.replace(4.  , np.nan, inplace=True)
d2.replace(5.  , np.nan, inplace=True)



d1[5] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

def etree_model(x, dropout=0.2):
    model = Sequential()
    model.add(Dense(x, input_dim=6, kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x*2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/2),  kernel_initializer='normal', activation='relu'))
    if dropout:
        model.add(Dropout(dropout))
    model.add(Dense(int(x/4),  kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


def err(model, x):
    emax = 1.
    emin = 1.
    for i in range(x):
        p = model.predict(X_train[i].reshape(1, 6))
        #print(p, Y_train[i], p/float(Y_train[i]))
        pp = Y_scaler.inverse_transform(p)
        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
        r = p/float(Y_train[i])
        if r > emax and py > 6.:
            emax = r
        if r < emin and py > 6.:
            emin = r          
        
        print(pp,py, r)
    return  emin , emax;


def process_model(model, batch_size=32, epochs=500):
    keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]
    history = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,
                        shuffle=True, verbose=0, validation_data=(X_test, Y_test), callbacks=keras_callbacks)
                
    train_score = model.evaluate(X_train, Y_train, verbose=0)
    valid_score = model.evaluate(X_test, Y_test, verbose=0)
    
bs=64
em = etree_model(93, dropout=0.2)
process_model(em, batch_size=64, epochs=5000)
print(err(em, 100))



Using TensorFlow backend.
W1009 15:58:58.494461 14316 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1009 15:58:58.548665 14316 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 15:58:58.555664 14316 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W1009 15:58:58.584738 14316 deprecation_wrapper.py:119] From C:\Users\Public\Documents\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_

[[8.003809]] [[8.23]] [[0.97788936]]
[[6.8378606]] [[7.08]] [[0.97333264]]
[[3.780297]] [[4.3]] [[0.9175074]]
[[5.934902]] [[5.75]] [[1.0238583]]
[[4.561679]] [[5.09]] [[0.9254835]]
[[2.3702507]] [[2.39]] [[0.9955013]]
[[3.287022]] [[4.26]] [[0.84457225]]
[[6.9117413]] [[7.]] [[0.9901934]]
[[0.63141644]] [[0.39]] [[1.101011]]
[[6.098653]] [[6.18]] [[0.9900553]]
[[5.6787024]] [[5.48]] [[1.0265645]]
[[4.330653]] [[4.11]] [[1.0361134]]
[[7.1890855]] [[6.66]] [[1.0610952]]
[[6.4227934]] [[6.43]] [[0.9991451]]
[[4.7498565]] [[4.18]] [[1.0922098]]
[[3.3959496]] [[3.33]] [[1.0123732]]
[[6.230529]] [[5.89]] [[1.0431595]]
[[3.80927]] [[3.78]] [[1.005064]]
[[3.7671485]] [[3.92]] [[0.97418046]]
[[6.2697825]] [[6.41]] [[0.98332727]]
[[8.604084]] [[8.63]] [[0.997562]]
[[1.5270306]] [[1.44]] [[1.0252995]]
[[6.621181]] [[6.94]] [[0.9643379]]
[[6.121757]] [[6.19]] [[0.99166745]]
[[5.589825]] [[6.]] [[0.9487281]]
[[9.398895]] [[9.19]] [[1.018668]]
[[1.3711888]] [[1.41]] [[0.9886185]]
[[6.3462124]] [[6.